Чтобы нам как-то измерить результаты модели, реализуем метрику accuracy разделив верно предсказанные классы на размер выборки

In [6]:
def score(y_true, y_pred):
    return sum(y_true == y_pred) / len(y_true)

## Классная работа № 5

В данной классной работе мы попробуем обучить несколько алгоритмов на известном датасете.

In [7]:
from sklearn import datasets

dataset = datasets.load_wine()

In [8]:
print(dataset['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

Согласно описанию датасета перед нами выборка 178 винных напитков, для которых мы будем предсказывать один из 3 классов.\
Для начала разделим выборку на обучающую и отложенную.

In [9]:
from sklearn.model_selection import train_test_split

X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

Ссылка на Google форму для проверки ответов: https://forms.gle/MfT4ZSibVKHgHDKi8

**1. Обучите на имеющейся выборке случайный лес (RandomForestClassifier) с деревьями максимальной глубины 1. Найдите точность (определяется с помощью функции `score`) у этого алгоритма.**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=1, random_state=17)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(f'Test score: {score(y_test, pred)}')

Test score: 0.9629629629629629


**2. Обучите на имеющейся выборке случайный лес (RandomForestClassifier) с деревьями максимальной глубины 2. Найдите точность (определяется с помощью функции `score`) у этого алгоритма.**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [16]:
model = RandomForestClassifier(max_depth=2, random_state=17)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(f'Test score: {score(y_test, pred)}')

Test score: 1.0


**3. С помощью `GridSearchCV` найдите наилучшие параметры из перечисленных ниже.**\
Используйте параметр `random_state=17` для воспроизводимости результатов.\
Максимальная глубина ограничена 1 у всех деревьев (параметр `max_depth` должен быть равен 1).

In [34]:
# params for grid search
rf_params = {'min_samples_leaf': list(range(1, 5)),
             'n_estimators': [1, 10, 20, 50, 100]}

from sklearn.model_selection import GridSearchCV

g_search_cv = GridSearchCV(estimator=RandomForestClassifier(max_depth=1, random_state=17, oob_score=True),
                           param_grid=rf_params,
                           error_score=score)               
g_search_cv.fit(X_train, y_train)
print(f'Best score: {g_search_cv.best_score_}')
print(f'Best params: {g_search_cv.best_params_}')

Best score: 0.9353333333333333
Best params: {'min_samples_leaf': 1, 'n_estimators': 100}


**4. Какой OOB score у модели с параметрами, подобранными в задании 3?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [23]:
model = RandomForestClassifier(max_depth=1, random_state=17, oob_score=True, min_samples_leaf=1, 
n_estimators=100)
model.fit(X_train, y_train)
model.oob_score_

0.9274193548387096

**5. Какой признак (порядковый номер) обладает наибольшей важностью у модели с параметрами, подобранными в задании 3?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.\
Для оценки важности надо воспользоваться impurity-based подходом.

In [36]:
import numpy as np

print(f'Best features: {g_search_cv.best_estimator_.feature_importances_}')
feature_importance = np.argmax(g_search_cv.best_estimator_.feature_importances_)
print(f'Best feature: {feature_importance}')
print(dataset.feature_names[feature_importance])

Best features: [0.13 0.   0.   0.04 0.03 0.06 0.15 0.   0.01 0.2  0.13 0.09 0.16]
Best feature: 9
color_intensity


**6. Какой точностью обладает алгоритм бэггинга на деревьях решений с максимальной глубиной 2?**\
Используйте параметр `random_state=17` для воспроизводимости результатов.

In [39]:
from sklearn.ensemble import BaggingClassifier
dt = DecisionTreeClassifier(max_depth = 2)
bc = BaggingClassifier(dt, random_state=17)
bc = bc.fit(X_train, y_train)
pred = model.predict(X_test)
print(f'Test score: {score(y_test, pred)}')

TypeError: __init__() got an unexpected keyword argument 'max_depth'